In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Импортируем нужные библиотеки**

In [ ]:
import numpy as np 
import pandas as pd 
import random

import matplotlib.pyplot as plt

import os
import zipfile

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

* Извлечение zip-файла

In [ ]:
local_zip = '/kaggle/input/dogs-vs-cats/train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
base_dir = '/tmp/train'
img_names = os.listdir(os.path.join(base_dir))
img_names[:10]

In [ ]:
sample = random.choice(img_names)
image = load_img("/tmp/train/"+sample)
plt.imshow(image)
plt.show()

**Подготовка данных**

In [ ]:
# установим метку собака и кошка для изображений
label = []

for i in img_names:
    if(i.split(".")[0] == "dog"):
        label.append("Dog")
    else:
        label.append("Cat")

label[:10]

In [ ]:
df = pd.DataFrame({"Image" : img_names, "Label" : label})
df.head()

In [ ]:
df.shape

In [ ]:
df.Label.value_counts().plot.bar(color = ['yellow','green'])
plt.show()

In [ ]:
train_df, validate_df = train_test_split(df, test_size = 0.2)
train_df = train_df.reset_index(drop = True)
validate_df = validate_df.reset_index(drop = True)

**Построим модель**

* Conv Layer: этот слой будет извлекать важные объекты из изображения
* Pooling Layer: этот слой уменьшает пространственный объем входного изображения после свертки, изолируя важные объекты
* Hidden Layer: Он соединяет сеть от одного слоя к другому слою
* Output Layer: это конечный слой с нейронами, равными количеству классов

In [ ]:
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'SGD',
              metrics = ['accuracy'])

In [ ]:
#Обучающий генератор с увеличением изображения
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "/tmp/train/", 
    x_col='Image',
    y_col='Label',
    target_size = (150, 150),
    class_mode = 'binary',
    batch_size = 20
)

In [ ]:
#validation generator
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/tmp/train/", 
    x_col='Image',
    y_col='Label',
    target_size = (150, 150),
    class_mode = 'binary',
    batch_size = 20
)

In [ ]:

local_zip = '/kaggle/input/dogs-vs-cats/test1.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

test_dir = '/tmp/test1/'
test_img = os.listdir(os.path.join(test_dir))
test_img[:10]

test_df = pd.DataFrame({'Image': test_img})

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "/tmp/test1/", 
    x_col = 'Image',
    y_col = None,
    class_mode = None,
    target_size = (150, 150),
    batch_size = 20,
    shuffle = False
)

In [ ]:
predict = model.predict_generator(test_generator, steps = np.ceil(12500/20))

In [ ]:
predict

In [ ]:
def label(predict):
    if(predict > 0.5):
        return "Dog"
    else:
        return "Cat"

In [ ]:
test_df['Label'] = predict
test_df['Label'] = test_df['Label'].apply(label)

In [ ]:
test_df.head()

In [ ]:
test_df.Label.value_counts().plot.bar(color = ['red','blue'])
plt.show()

In [ ]:
v = random.randint(0, 12000)

sample_test = test_df.iloc[v:(v+18)].reset_index(drop = True)
sample_test.head()

plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['Image']
    category = row['Label']
    img = load_img("/tmp/test1/" + filename, target_size = (150, 150))
    plt.subplot(6, 3, index + 1)
    plt.imshow(img)
    plt.xlabel(filename + ' ( ' + "{}".format(category) + ' )' )
plt.tight_layout()
plt.show()